In [1]:
from simulator.bb84 import simulation_bb84
from tqdm import tqdm
from numpy import arange
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import genetic_algorithm.genetic_algorithm

# Preliminary tests without optimisation

In [12]:
# Create an empty list to store the dictionaries
results_list = []

for i in range(20):
    
    for qber in tqdm(arange(start=0.01, stop=0.16, step=0.01)):
        simulation_results = simulation_bb84(
            alice_basis_length=10000,
            disturbance_probability=qber,
            cascade_n_passes=4
        )
        
        # Append the results as a new dictionary to the list
        new_row = {
            'qber': qber,
            'final error rate': simulation_results.get('error rate'),
            'error estimate': simulation_results.get('error estimate'),
            'initial key length': simulation_results.get('key length history').get('qubits'),
            'key length after sifting': simulation_results.get('key length history').get('sifting'),
            'key length after err est': simulation_results.get('key length history').get('error estimation'),
            'key length after err corr': simulation_results.get('key length history').get('error correction')
        }
        results_list.append(new_row)

# Create a DataFrame from the list of dictionaries
results_df = pd.DataFrame(results_list)

In [13]:
results_df.sample(10)

Plotting results:

In [21]:
# Create a 3D scatter plot
fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(111, projection='3d')

# Prepare data for scatter plot
initial_params = results_df['qber']
stages = np.arange(1, 4)  # Assuming there are three stages in the simulation

# Scatter plot for Variable_1
ax.scatter(
    initial_params, 
    np.full_like(initial_params, 1), 
    results_df['key length after sifting'], 
    label='after sifting'
)

# Scatter plot for Variable_2
ax.scatter(
    initial_params, 
    np.full_like(initial_params, 2), 
    results_df['key length after err est'], 
    label='after error estimation'
)

# Scatter plot for Variable_3
ax.scatter(
    initial_params, 
    np.full_like(initial_params, 3), 
    results_df['key length after err corr'], 
    label='after error correction'
)

# Set labels for axes
ax.set_xlabel('QBER')
ax.set_ylabel('Simulation Stages')
ax.set_zlabel('Key length')

# Set plot title
plt.title('Key lengths at given stages of the BB84 simulation')

# Set the viewing point
ax.view_init(elev=20, azim=60)  # Adjust these values to change the viewing point

# Show the legend
ax.legend()

# Show the plot
plt.show()

In [31]:
# Create a 3D scatter plot
fig = plt.figure(figsize=(12, 8))
ax_3d = fig.add_subplot(111, projection='3d')

# Plot the 3D scatter plot
ax_3d.scatter(results_df['qber'], results_df['error estimate'], results_df['key length after err corr'], c='blue', label='3D scatter')

# Set labels for axes
ax_3d.set_xlabel('QBER')
ax_3d.set_ylabel('Error estimation')
ax_3d.set_zlabel('Key length after error correction')

# Set plot title
plt.title('Key lengths for different QBER and its estimates')

# Set the viewing point
ax_3d.view_init(elev=20, azim=60)  # Adjust these values to change the viewing point

# Plot projections on XY, XZ, and YZ planes
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# XY projection
axes[0].scatter(results_df['qber'], results_df['error estimate'], c='red', label='XY projection')
axes[0].set_xlabel('QBER')
axes[0].set_ylabel('Error estimation')
axes[0].set_title('XY Projection')

# XZ projection
axes[1].scatter(results_df['qber'], results_df['key length after err corr'], c='green', label='XZ projection')
axes[1].set_xlabel('QBER')
axes[1].set_ylabel('Key length after error correction')
axes[1].set_title('XZ Projection')

# YZ projection
axes[2].scatter(results_df['error estimate'], results_df['key length after err corr'], c='purple', label='YZ projection')
axes[2].set_xlabel('Error estimation')
axes[2].set_ylabel('Key length after error correction')
axes[2].set_title('YZ Projection')

# Show the plots
plt.show()

In [37]:
results_df.to_csv("new_results.csv")

# Simulation with no error estimation, just as in the '93 paper

In [11]:
# Create an empty list to store the dictionaries
results93_list = []

for i in range(20):
    
    for qber in tqdm(arange(start=0.01, stop=0.16, step=0.01)):
        simulation_results = simulation_bb84(
            alice_basis_length=10000,
            disturbance_probability=qber,
            cascade_n_passes=4,
            error_estimation='none'
        )
        
        # Append the results as a new dictionary to the list
        new_row = {
            'qber': qber,
            'final error rate': simulation_results.get('error rate'),
            'error estimate': simulation_results.get('error estimate'),
            'initial key length': simulation_results.get('key length history').get('qubits'),
            'key length after sifting': simulation_results.get('key length history').get('sifting'),
            'key length after err est': simulation_results.get('key length history').get('error estimation'),
            'key length after err corr': simulation_results.get('key length history').get('error correction'),
            'no. cascade pass.': simulation_results.get('key length history').get('no. cascade pass.')
        }
        results93_list.append(new_row)

# Create a DataFrame from the list of dictionaries
results93_df = pd.DataFrame(results93_list)

In [12]:
results93_df.sample(10)

In [7]:
# Create a 3D scatter plot
fig = plt.figure(figsize=(12, 8))
ax_3d = fig.add_subplot(111, projection='3d')

# Plot the 3D scatter plot
ax_3d.scatter(results93_df['qber'], results93_df['error estimate'], results93_df['key length after err corr'], c='blue', label='3D scatter')

# Set labels for axes
ax_3d.set_xlabel('QBER')
ax_3d.set_ylabel('Error estimation')
ax_3d.set_zlabel('Key length after error correction')

# Set plot title
plt.title('Key lengths for different QBER - no error estimation')

# Set the viewing point
ax_3d.view_init(elev=20, azim=60)  # Adjust these values to change the viewing point

# Plot projections on XY, XZ, and YZ planes
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# XY projection
axes[0].scatter(results93_df['qber'], results93_df['error estimate'], c='red', label='XY projection')
axes[0].set_xlabel('QBER')
axes[0].set_ylabel('Error estimation')
axes[0].set_title('XY Projection')

# XZ projection
axes[1].scatter(results93_df['qber'], results93_df['key length after err corr'], c='green', label='XZ projection')
axes[1].set_xlabel('QBER')
axes[1].set_ylabel('Key length after error correction')
axes[1].set_title('XZ Projection')

# YZ projection
axes[2].scatter(results93_df['error estimate'], results93_df['key length after err corr'], c='purple', label='YZ projection')
axes[2].set_xlabel('Error estimation')
axes[2].set_ylabel('Key length after error correction')
axes[2].set_title('YZ Projection')

# Show the plots
plt.show()

Perhaps it depends on whether all 4 CASCADE passes were done? What's the final error rate?

In [8]:
plt.scatter(results93_df['final error rate'], results93_df['key length after err corr'])

In [10]:
plt.scatter(results93_df['qber'], results93_df['final error rate'])

In [ ]:
plt.scatter(results93_df['no. cascade pass'], results93_df['key length after err corr'])

# Optimisation with a classical genetic algorithm

In [ ]:
gene_space = [0, 1, 2, 3, 4]
num_generations = 100
population_size = 50
elite_size = 2

In [ ]:
def genome_generator(args):
    """Function for randomising BB84 input parameters"""
    
    return genome